 # Multi Query Retrievers
 This tutorial demonstrates how to build and use multi-query retrievers with LangChain.
 We'll start by loading documents, splitting them into chunks, and using embeddings
 to enable similarity search and retrieval.

In [15]:
from langchain.storage import InMemoryByteStore
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from rich import print
import uuid

 ## Step 1: Load Documents
 Here, we load the documents using `TextLoader`. You can replace the file paths
 with your own document paths.

In [16]:
loaders = [
    TextLoader("data/langchain.md"),
    TextLoader("data/langchain2.md"),
]
docs = []
for loader in loaders:
    docs.extend(loader.load())

 ## Step 2: Split Documents into Large Chunks
 Using `RecursiveCharacterTextSplitter`, we split documents into smaller chunks
 for more efficient processing and retrieval.

In [32]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000)
docs = text_splitter.split_documents(docs)



 ## Step 3: Initialize the Vectorstore
 The vectorstore indexes the document chunks for similarity search. We use the `Chroma`
 library with embeddings from Ollama.

In [33]:
vectorstore = Chroma(
    collection_name="full_documents",
    embedding_function=OllamaEmbeddings(model='snowflake-arctic-embed:33m')
)



 ## Step 4: Create Smaller Chunks
 For fine-grained retrieval, we split the documents further into smaller chunks
 and associate metadata for linking with original documents.

In [34]:
from langchain.retrievers.multi_vector import MultiVectorRetriever

# Initialize storage for parent documents
store = InMemoryByteStore()
id_key = "doc_id"

# Create the retriever
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    byte_store=store,
    id_key=id_key,
)

doc_ids = [str(uuid.uuid4()) for _ in docs]

# Split into smaller chunks
child_text_splitter = RecursiveCharacterTextSplitter(chunk_size=400)
sub_docs = []
for i, doc in enumerate(docs):
    _id = doc_ids[i]
    _sub_docs = child_text_splitter.split_documents([doc])
    for _doc in _sub_docs:
        _doc.metadata[id_key] = _id
    sub_docs.extend(_sub_docs)



 ## Step 5: Add Documents to Vectorstore
 Add the smaller chunks and their metadata to the vectorstore for similarity search.

In [35]:
retriever.vectorstore.add_documents(sub_docs)
retriever.docstore.mset(list(zip(doc_ids, docs)))
print(retriever)

MultiVectorRetriever(
    vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x30d170710>,
    byte_store=<langchain_core.stores.InMemoryByteStore object at 0x30e3df550>,
    docstore=<langchain.storage.encoder_backed.EncoderBackedStore object at 0x30d1f28d0>,
    search_kwargs={}
)



 ## Step 6: Perform a Similarity Search
 Use the retriever to find documents similar to the query "LangChain".

In [36]:
print(retriever.vectorstore.similarity_search("LangChain")[0:2])

[
    Document(
        id='58b2804b-7b53-40ce-97d9-ede3e0852911',
        metadata={'doc_id': '77eeb4f3-f7e8-4df2-a16b-3e63d3978eb2', 'source': 'data/langchain.md'},
        page_content='LangChain provides a consistent interface for working with chat models from different 
providers while offering additional features for monitoring, debugging, and optimizing the performance of 
applications that use LLMs.'
    ),
    Document(
        id='97524e7b-902d-4069-a3ce-06f873e0491f',
        metadata={'doc_id': 'a5154e19-f37d-4514-aaba-68933f778c71', 'source': 'data/langchain.md'},
        page_content='LangChain provides a consistent interface for working with chat models from different 
providers while offering additional features for monitoring, debugging, and optimizing the performance of 
applications that use LLMs.'
    )
]



 ## Step 7: Multi-Modal Retrieval
 Modify the search type to use Maximal Marginal Relevance (MMR) for diverse results.

In [37]:
from langchain.retrievers.multi_vector import SearchType

retriever.search_type = SearchType.mmr
print(retriever.invoke("LangChain")[0:2])

[]



 ## Step 8: Summarize Documents
 Associate summaries with documents using a language model.

In [38]:
import getpass
import os
from langchain_openai import ChatOpenAI
from langchain_ollama import ChatOllama

# Initialize LLM
llm = ChatOllama(model='llama3.2:1b')

from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

chain = (
    {"doc": lambda x: x.page_content}
    | ChatPromptTemplate.from_template("Summarize the following document:\n\n{doc}")
    | llm
    | StrOutputParser()
)

# Summarize documents in parallel
summaries = chain.batch(docs, {"max_concurrency": 5})
print(summaries[0:2])

[
    'The document provides an overview of LangChain, specifically the concept of large language models (LLMs) and 
their application in a chat model interface. It mentions that LLMs are machine learning models that excel in 
various language-related tasks without requiring task-specific fine-tuning for every scenario. The document 
highlights the capabilities of modern LLMs, including text generation, translation, summarization, question 
answering, etc., which can be accessed through a chat model interface.',
    'The document discusses the advancements in chat models, specifically highlighting the following 
capabilities:\n\n1. **Tool calling**: A native API that allows LLMs to interact with external services, APIs, and 
databases.\n2. **Structured output**: The ability of a chat model to respond in a structured format, such as JSON, 
matching a given schema.\n3. **Multimodality**: Working with data beyond text, including images, audio, and 
video.\n\nThese features enable developers to build more advanced and complex applications that can handle various 
types of input and output.'
]



 ## Step 9: Add Summaries to Vectorstore
 Store the summaries in the vectorstore for enhanced retrieval capabilities.

In [39]:
vectorstore = Chroma(collection_name="summaries", embedding_function=OpenAIEmbeddings())
store = InMemoryByteStore()
id_key = "doc_id"

retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    byte_store=store,
    id_key=id_key,
)

summary_docs = [
    Document(page_content=s, metadata={id_key: doc_ids[i]})
    for i, s in enumerate(summaries)
]

retriever.vectorstore.add_documents(summary_docs)
retriever.docstore.mset(list(zip(doc_ids, docs)))



 ## Step 10: Retrieve Summaries
 Perform a similarity search on the summaries to get the most relevant results.

In [40]:
sub_docs = retriever.vectorstore.similarity_search("LangChain")
print("Sub Docs:\n",sub_docs[0])

retrieved_docs = retriever.invoke("LangChain")
print("Retrieved Docs:\n",retrieved_docs[0:2])
# len(retrieved_docs[0].page_content)

Sub Docs:

Document(
    id='810f2212-0166-4293-a9d3-ddf1834072d8',
    metadata={'doc_id': 'a5154e19-f37d-4514-aaba-68933f778c71'},
    page_content="The document describes LangChain's features as follows:\n\n- Provides a consistent interface for 
interacting with various chat models.\n- Offers additional features such as monitoring, debugging, and optimization
for improving application performance with Large Language Models (LLMs)."
)

Retrieved Docs:

[
    Document(
        metadata={'source': 'data/langchain.md'},
        page_content='Features[\u200b](https://python.langchain.com/docs/concepts/chat_models/#features "Direct 
link to 
Features")\n-------------------------------------------------------------------------------------------------------
\n\nLangChain provides a consistent interface for working with chat models from different providers while offering 
additional features for monitoring, debugging, and optimizing the performance of applications that use LLMs.'
    )
]